In [ ]:
import os
import sys
import time

from biokb_chebi.db.importer import DatabaseImporter
from biokb_chebi.neo4j_importer import Neo4jImporter
from biokb_chebi.rdf.turtle import TurtleCreator
from biokb_chebi.manager import MySQLDatabase

sys.path.append(os.path.join("..", "tests"))

from docker_compose_defaults import (  # type: ignore
    MYSQL_DATABASE_TEST,
    MYSQL_HOST_TEST,
    MYSQL_PASSWORD_TEST,
    MYSQL_PORT_TEST,
    MYSQL_USER_TEST,
    NEO4J_PASSWD_TEST,
    NEO4J_URI_TEST,
    NEO4J_USER_TEST,
)

# Test biokb_chebi with test data

In [ ]:
# MySQL Database
mysql_db = MySQLDatabase(
    user=MYSQL_USER_TEST,
    password=MYSQL_PASSWORD_TEST,
    host=MYSQL_HOST_TEST,
    port=MYSQL_PORT_TEST,
    db_name=MYSQL_DATABASE_TEST,
)

start_and_stop_podman_compose = True

if start_and_stop_podman_compose:
    command = "podman-compose -f ../docker-compose-test.yml up -d neo4j mysql "
    os.popen(command).read()
    time.sleep(5)

test_data_folder = os.path.join("..", "tests", "data")

## Import data into MySQL

In [ ]:
importer = DatabaseImporter(engine=mysql_db.engine, data_folder=test_data_folder)
importer.import_db()

## Create ttls

In [ ]:
turtle_creator = TurtleCreator(
    engine=mysql_db.engine, export_to_folder=test_data_folder
)
path_to_zipped_ttls = turtle_creator.create_all_ttls()
path_to_zipped_ttls

## Import turtle files into Neo4J

In [ ]:
neo4j_importer = Neo4jImporter(
    neo4j_pwd=NEO4J_PASSWD_TEST,
    neo4j_user=NEO4J_USER_TEST,
    neo4j_uri=NEO4J_URI_TEST,
    delete_existing_graph=True,
    path_or_url=path_to_zipped_ttls,
)
neo4j_importer.import_ttls()

In [ ]:
if start_and_stop_podman_compose:
    command = "podman-compose -f ../docker-compose-test.yml stop mysql neo4j"
    os.popen(command).read()

## Check the imports

1. Start MySQL, Neo4J and phpMyAdmin with `podman-compose -f docker-compose-test.yml up -d` from the project root folder in the terminal
2. Open [PhpMyAdmin](http://localhost:8082) with
   1. *user*: biokb_chebi_user
   2. *password*: biokb_chebi_password
3. Open [Neo4J Browser](http://localhost:7476/browser/) and login with ...
   - *Connect URL*: neo4j://localhost:7689
   - *Authentication type*: Username/Password
   - *Username*: neo4j
   - *Password*: neo4j_password